### Задание №1

Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:   
x - последовательность цифр.  
y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10.  

Задача:   
научить модель предсказывать y(i) по x(i).
пробовать RNN, LSTM, GRU.  

In [1]:
import torch as tr
import copy
import time
import torch
from torch import nn
import re
import random
import tqdm
import pandas as pd
import os
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
dev = tr.device('cuda' if tr.cuda.is_available() else 'cpu')
if dev.type == 'cuda': print('work on', tr.cuda.get_device_name(0))
else: print('work on CPU')

work on Tesla T4


In [10]:
def f(x):
  return (x + x[:,0:1]) % 10

t = tr.tensor
res = []

In [11]:
class rnn1(tr.nn.Module):
    def __init__(self, n_classes, embedding_size, hidden_size, num_layers, rnn_type):
        super().__init__()
        self.emb = tr.nn.Embedding(n_classes, embedding_size)
        self.rnn = rnn_type(embedding_size, hidden_size, num_layers, batch_first = True)
        self.lin = tr.nn.Linear(hidden_size, n_classes)
    def forward(self, input):
        r = self.emb(input)
        r = self.rnn(r)[0]
        r = self.lin(r).squeeze()
        return r


In [12]:
n_iters        = 1000
n_classes      = 10
embedding_size = 10
hidden_size    = 50
num_layers     = 1
batch_size     = 100
seq_len        = 100
loss_best      = 10**10
learn_rate     = 0.0025
rnn_list      = [tr.nn.RNN, tr.nn.LSTM, tr.nn.GRU]

In [16]:
s_l_list = [20,50,100,125,150,200]
df_ind   = [x.__name__ for x in rnn_list]

df_2  = pd.DataFrame(index = df_ind, columns = s_l_list)

for i in s_l_list:
    for rnn_type in rnn_list:
        model = rnn1(n_classes, embedding_size, hidden_size, num_layers, rnn_type).to(dev)
        opt   = tr.optim.Adam(model.parameters(), lr = 0.0015)
        crit  = tr.nn.CrossEntropyLoss()

        for iter in range(n_iters):
            model.train()
            x = tr.randint(n_classes, (batch_size, i)).to(dev)
            y = f(x)
            opt.zero_grad()
            pred = model(x)
            score = (y == pred.argmax(dim=-1)).sum() / t(y.shape).prod()
            loss = crit(pred.reshape(-1, n_classes), y.flatten()).to(dev)  #
            if loss < loss_best:
                model_best = copy.copy(model)
                loss_best = loss
            loss.backward()
            opt.step()

        model.eval()
        x = tr.randint(n_classes, (batch_size, i)).to(dev)
        y = f(x)
        pred = model(x).argmax(dim=-1)
        score = (y == pred).sum() / t(y.shape).prod()
        df_2.loc[rnn_type.__name__, i] = round(score.item(),5)
df_2

,20,50,100,125,150,200
RNN,0.522,0.2228,0.1082,0.10952,0.10373,0.10475
LSTM,1.0,1.0,0.5189,0.6068,0.1746,0.10565
GRU,1.0,1.0,0.5604,0.7944,0.99393,0.10165


In [17]:
seq_len  = 100
n_l_list = [1, 3, 5, 7]

df_1  = pd.DataFrame(index = df_ind, columns = n_l_list)


for i in n_l_list:
    for rnn_type in rnn_list:
        model = rnn1(n_classes, embedding_size, hidden_size, i, rnn_type).to(dev)
        opt   = tr.optim.Adam(model.parameters(), lr = 0.0015)
        crit  = tr.nn.CrossEntropyLoss()

        for iter in range(n_iters):
            model.train()
            x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
            y = f(x)
            opt.zero_grad()
            pred = model(x)
            score = (y == pred.argmax(dim=-1)).sum() / t(y.shape).prod()
            loss = crit(pred.reshape(-1, n_classes), y.flatten()).to(dev)  #
            if loss < loss_best:
                model_best = copy.copy(model)
                loss_best = loss
            loss.backward()
            opt.step()

        model.eval()
        x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
        y = f(x)
        pred = model(x).argmax(dim=-1)
        score = (y == pred).sum() / t(y.shape).prod()
        df_1.loc[rnn_type.__name__, i] = round(score.item(),5)

df_1

,1,3,5,7
RNN,0.1049,0.207,0.3849,0.1989
LSTM,0.5332,0.7908,0.4903,0.1128
GRU,0.9302,1.0,0.6706,0.1535


In [19]:
num_layers     = 1
h_s_list       = [50,75,100,125,150,200]

df_4  = pd.DataFrame(index = df_ind, columns = h_s_list)

for i in h_s_list:
    for rnn_type in rnn_list:
        model = rnn1(n_classes, embedding_size, i, num_layers, rnn_type).to(dev)
        opt   = tr.optim.Adam(model.parameters(), lr = 0.0015)
        crit  = tr.nn.CrossEntropyLoss()

        for iter in range(n_iters):
            model.train()
            x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
            y = f(x)
            opt.zero_grad()
            pred = model(x)
            score = (y == pred.argmax(dim=-1)).sum() / t(y.shape).prod()
            loss = crit(pred.reshape(-1, n_classes), y.flatten()).to(dev)  #
            if loss < loss_best:
                model_best = copy.copy(model)
                loss_best = loss
            loss.backward()
            opt.step()

        model.eval()
        x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
        y = f(x)
        pred = model(x).argmax(dim=-1)
        score = (y == pred).sum() / t(y.shape).prod()
        df_4.loc[rnn_type.__name__, i] = round(score.item(),5)

df_4

,50,75,100,125,150,200
RNN,0.1002,0.1106,0.1199,0.119,0.1099,0.1088
LSTM,0.324,0.4958,0.5093,1.0,0.6066,1.0
GRU,0.9947,0.9972,1.0,1.0,1.0,1.0


In [20]:

hidden_size = 50
emb_s_list  = [5,10,20,30,50]

df_3  = pd.DataFrame(index = df_ind, columns = emb_s_list)

for i in emb_s_list:
    for rnn_type in rnn_list:
        model = rnn1(n_classes, i, hidden_size, num_layers, rnn_type).to(dev)
        opt   = tr.optim.Adam(model.parameters(), lr = 0.0015)
        crit  = tr.nn.CrossEntropyLoss()

        for iter in range(n_iters):
            model.train()
            x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
            y = f(x)
            opt.zero_grad()
            pred = model(x)
            score = (y == pred.argmax(dim=-1)).sum() / t(y.shape).prod()
            loss = crit(pred.reshape(-1, n_classes), y.flatten()).to(dev)  #
            if loss < loss_best:
                model_best = copy.copy(model)
                loss_best = loss
            loss.backward()
            opt.step()

        model.eval()
        x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
        y = f(x)
        pred = model(x).argmax(dim=-1)
        score = (y == pred).sum() / t(y.shape).prod()
        df_3.loc[rnn_type.__name__, i] = round(score.item(),5)

df_3

,5,10,20,30,50
RNN,0.109,0.0789,0.1093,0.1124,0.1067
LSTM,0.5069,0.4985,0.513,0.759,0.9322
GRU,0.5244,0.7229,0.9998,0.9859,0.8435


In [21]:
h_s_list  = [50,70,90,120]
n_l_list  = [1,2,3,4]

df_5  = pd.DataFrame(index = n_l_list, columns = h_s_list)

for i in h_s_list:
    for j in n_l_list:
        model = rnn1(n_classes, embedding_size, i, j, tr.nn.GRU).to(dev)
        opt   = tr.optim.Adam(model.parameters(), lr = 0.0005)
        crit  = tr.nn.CrossEntropyLoss()

        for iter in range(n_iters):
            model.train()
            x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
            y = f(x)
            opt.zero_grad()
            pred = model(x)
            score = (y == pred.argmax(dim=-1)).sum() / t(y.shape).prod()
            loss = crit(pred.reshape(-1, n_classes), y.flatten()).to(dev)  #
            if loss < loss_best:
                model_best = copy.copy(model)
                loss_best = loss
            loss.backward()
            opt.step()

        model.eval()
        x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
        y = f(x)
        pred = model(x).argmax(dim=-1)
        score = (y == pred).sum() / t(y.shape).prod()
        df_5.loc[j, i] = round(score.item(),5)

df_5

,50,70,90,120
1,0.1095,0.13,0.1075,0.1988
2,0.1039,0.2247,0.3974,0.5125
3,0.1106,0.1392,0.4327,0.7828
4,0.1049,0.2975,0.535,0.5605


Выводы:
1. увеличение размера входного тензора не повышает качество модели.  
2. увеличение количества nn слоев до определенного уровня может повысить качество модели.  
3. увеличение размера скрытого слоя часто повышает качество модели.  
4. увеличение размера эмбеддинга увеличивает качество модели GRU до определенного предела